# INF8215 - Intelligence artif.: méthodes et algorithmes 
## Automne 2019 - TP2 - Recherche Adversarielle 
### Membres de l'équipe
- El-Asmar, Jad (1853357)
- Elakhrass, Mohamed Ali (1847744)
- Hasrouni, Elio (1847193)




## Directives de remise
Le travail sera réalisé avec la  même équipe que pour le TP1. Vous remettrez ce fichier nommé TP2\_NomDuMembre1\_NomDuMembre2\_NomDuMembre3.ipynb dans la boîte de remise sur moodle. 

Tout devra être remis avant le **6 novembre à 23h55**. Tout travail en retard sera pénalisé
d’une valeur de 10\% par jour de retard.

## Barème
Partie 1: 25 points

Partie 2: 35 points

Partie 3: 15 points

Partie 4: 15 points

Partie 5: 10 points

Pour un total de 100 points possibles sur 100 points.


## Recherche Adversarielle

La recherche adversarielle est une méthode d’intelligence artificielle qui est utilisée pour prédire les actions d’un adversaire dans un jeu vidéo. Il consiste à construire un arbre avec les actions possibles suivi des réactions de l’adversaire à ces actions. Pour choisir l’action optimale, il existe plusieurs méthodes. Une très commune est la recherche minimax. Elle évalue la valeur des actions et maximise la valeur de celle du joueur et minimise la valeur de celle de l’adversaire.

### Rush Hour

Vous avez vu le jeu de puzzle, Rush Hour, dans le TP précédent. Dans le cadre de ce TP, un adversaire a été ajouté pour transformer Rush Hour en jeu adversarielle. À chaque tour, l’adversaire ajoute une roche dans la grille. Cette roche disparaît après 1 tour et empêche les voitures de se déplacer. Son but est d’obliger le joueur à faire le plus de coups possible et de l’empêcher de bouger complètement. Le joueur peut déplacer une voiture, selon les règles vues dans le tp précédent, par tour. Cette version de Rush Hour est un jeu zero-sum déterministe.

![Rushhour](https://i.stack.imgur.com/NzztF.jpg)

### But

Votre but dans ce TP est de développer un algorithme minimax pour permettre à votre AI de terminer le niveau de Rush Hour **avec un minimun de mouvement**. Une version complétée des classes Rush Hour et State est fournie pour vous aider à faire le TP. Ces classes vont être réutilisées pour représenter les états à évaluer. Vous pouvez les remplacer par vos propres implémentations en Python (vous pouvez aussi réécrire le TP dans le langage de votre choix).  

## 1. Implémentation de Rush Hour (25pts)
Les classes suivantes sont complètes pour représenter les différents états d’une partie de Rush Hour normale.
Il manque la représentation des coups de l’adversaire : vous auriez à implémenter cette partie-là dans les classes touchées.

L’état du stationnement au début et à tout moment du jeu est représenté par un objet de la classe **State** qui contient les champs suivants : 

- `pos` : un vecteur indiquant la position de chaque véhicule sur la grille (la valeur représente la première case occupée par la voiture);

- `c` : l’index de la voiture déplacée pour atteindre l’état courant à partir de l’état précédent;

- `d` : la direction du déplacement fait à partir de l’état précédent pour atteindre l’état courant (+1 : vers la droite ou vers le bas, -1 : vers la gauche ou vers le haut);

- `prev` : l’état précédent

- `score` : le score associé à l'état

- `rock` : la position de la roche dans la grille (ligne,colonne)

In [2]:
import copy
import math
import numpy as np
import random

from collections import deque

In [3]:
class State:
    """
    Contructeur d'un état initial
    """

    def __init__(self, pos):
        """
        pos donne la position de la voiture i dans sa ligne ou colonne (première case occupée par la voiture);
        """
        self.pos = np.array(pos)

        """
        c,d et prev premettent de retracer l'état précédent et le dernier mouvement effectué
        """
        self.c = self.d = self.prev = None

        self.nb_moves = 0
        self.score = 0

        # TODO
        self.rock = None

    """
    Constructeur d'un état à partir du mouvement (c,d)
    """
    def move(self, c, d):
        s = State(self.pos)
        s.prev = self
        s.pos[c] += d
        s.c = c
        s.d = d
        s.nb_moves = self.nb_moves + 1
        s.rock = s.prev.rock
        # TODO
        return s

    def put_rock(self, rock_pos):
        # TODO
        s = copy.deepcopy(self)
        s.rock = rock_pos

        return s

    def score_state(self, rh):
        self.score = self.estimee(rh)

        return self.score

    def success(self):
        return self.pos[0] == 4

    def estimee(self, rh):
        # Gain de 100 si l'auto rouge s'approche de la sortie
        cost = 25 * self.pos[0]
        # Penalite si une auto bloque l'auto rouge
        cost -= self.cars_front_red(rh)
        # Penalite si une auto est bloque (aucun mouvement possible)
        cost -= self.car_blocked(rh)
        # Gain si une auto horizontale est a gauche de l'auto rouge
        cost += self.cars_left_red(rh)

        return cost

    def cars_front_red(self, rh):
        penalty = 0

        for i in range(1, rh.nbcars):
            if not rh.horiz[i] and rh.move_on[i] >= self.pos[0] + rh.length[0] and self.pos[i] <= 2 <= \
                    (self.pos[i] + rh.length[i] - 1):
                if rh.length[i] == 3:
                    penalty += ((3 - self.pos[i]) * rh.move_on[i] * 2)
                    for j in range(self.pos[i] + rh.length[i], 6):
                        if not rh.free_pos[j][rh.move_on[i]]:
                            penalty += (rh.move_on[i] * self.blocking_car(j, rh.move_on[i], rh))
                else:
                    penalty += (rh.move_on[i] * 2)
                    if self.pos[i] == 1:
                        if not rh.free_pos[0][rh.move_on[i]]:
                            penalty += (rh.move_on[i] * self.blocking_car(0, rh.move_on[i], rh))
                    elif self.pos[i] == 2:
                        if not rh.free_pos[4][rh.move_on[i]]:
                            penalty += (rh.move_on[i] * self.blocking_car(4, rh.move_on[i], rh))

        return penalty

    def blocking_car(self, line, col, rh):
        for i in range(1, rh.nbcars):
            if rh.horiz[i] and rh.move_on[i] == line and self.pos[i] <= col <= self.pos[i] + rh.length[i] - 1:
                if rh.length[i] == 3:
                    if col >= 3:
                        return self.pos[i]
                    else:
                        return 3 - self.pos[i]
                else:
                    return 1

        # The rock is blocking
        return 1

    def car_blocked(self, rh):
        penalty = 0

        possible_moves = rh.possible_moves(self)
        car_moved = [move.c for move in possible_moves]
        for i in range(1, rh.nbcars):
            if i not in car_moved:
                penalty += (rh.move_on[i] + 1)

        return penalty

    def cars_left_red(self, rh):
        gain = 0
        for i in range(1, rh.nbcars):
            if rh.horiz[i] and (self.pos[i] + rh.length[i] - 1) < (self.pos[0]):
                gain += 10
            elif rh.horiz[i] and (self.pos[i] + rh.length[i]) <= (
                    self.pos[0] + rh.length[0]):  # Si t a gauche de l'auto rouge
                gain += 5

        return gain

    def __eq__(self, other):
        if not isinstance(other, State):
            return NotImplemented
        if len(self.pos) != len(other.pos):
            print("les états n'ont pas le même nombre de voitures")

        return np.array_equal(self.pos, other.pos)

    def __hash__(self):
        h = 0
        for i in range(len(self.pos)):
            h = 37 * h + self.pos[i]
        return int(h)

    def __lt__(self, other):
        return self.score < other.score

    def __add__(self, other):
        return other.score + self.score

La représentation du problème est faite par la classe **Rushhour** qui contient les champs suivants :

- `nbcars` : le nombre de voitures dans la grille;
- `horiz` : un vecteur contenant un bool indiquant si la voiture bouge horizontalement ou pas;
- `length`: un vecteur contenant la longueur de chaque voiture (2 ou 3);
- `move_on` : un vecteur contenant le numéro de la ligne ou la colonne où se trouve la voiture (0-5);
- `color` : un vecteur contenant le string indiquant la couleur de chaque voiture;
- `free_pos` : une matrice 6x6 contenant un bool permettant de savoir si la case est libre

Toutes les informations pour une voiture se trouvent dans le même index dans chacun des vecteurs.

La fonction *`print_pretty_grid()`* sert à imprimer la grille en utilisant la première lettre des couleurs pour vous aider à mieux visualiser le problème.

In [4]:
class Rushhour:

    def __init__(self, horiz, length, move_on, color=None):
        self.nbcars = len(horiz)
        self.horiz = horiz
        self.length = length
        self.move_on = move_on
        self.color = color

        self.free_pos = None

    def init_positions(self, state):
        self.free_pos = np.ones((6, 6), dtype=bool)
        for i in range(self.nbcars):
            if self.horiz[i]:
                self.free_pos[self.move_on[i], state.pos[i]:state.pos[i] + self.length[i]] = False
            else:
                self.free_pos[state.pos[i]:state.pos[i] + self.length[i], self.move_on[i]] = False
        # TODO
        if state.rock is not None:
            self.free_pos[state.rock[0], state.rock[1]] = False

    def possible_moves(self, state):
        self.init_positions(state)
        new_states = []
        for i in range(self.nbcars):
            if self.horiz[i]:
                if state.pos[i] + self.length[i] - 1 < 5 and self.free_pos[self.move_on[i], state.pos[i] + self.length[i]]:
                    new_states.append(state.move(i, +1))
                if state.pos[i] > 0 and self.free_pos[self.move_on[i], state.pos[i] - 1]:
                    new_states.append(state.move(i, -1))
            else:
                if state.pos[i] + self.length[i] - 1 < 5 and self.free_pos[state.pos[i] + self.length[i], self.move_on[i]]:
                    new_states.append(state.move(i, +1))
                if state.pos[i] > 0 and self.free_pos[state.pos[i] - 1, self.move_on[i]]:
                    new_states.append(state.move(i, -1))
        return new_states

    def possible_rock_moves(self, state):
        self.init_positions(state)
        new_states = []
        # TODO
        state_temp = state
        for i in range(0, 6):
            for j in range(0, 6):
                if self.free_pos[i][j]:
                    if state.rock is not None:
                        if state.rock[1] != j and state.rock[0] != i and i != 2:
                            new_states.append(state_temp.put_rock((i, j)))
                    else:
                        if i != 2:
                            new_states.append(state_temp.put_rock((i, j)))

        return new_states

    def print_pretty_grid(self, state):
        self.init_positions(state)
        grid = np.chararray((6, 6))
        grid[:] = '-'
        for car in range(self.nbcars):
            for pos in range(state.pos[car], state.pos[car] + self.length[car]):
                if self.horiz[car]:
                    grid[self.move_on[car]][pos] = self.color[car][0]
                else:
                    grid[pos][self.move_on[car]] = self.color[car][0]
        if state.rock is not None:
            grid[state.rock] = 'x'
        print(grid)

### 1.1 Implémentation

1. Modifier la classe State pour qu'elle contienne la position de la roche ainsi que la fonction *`put_rock()`* pour ajouter une nouvelle roche et enlever l'ancienne. Les roches sont représentées par un tuple (ligne,colonne). La fonction retourne un nouvel objet State.

2. Modifier la fonction *`init_position()`* de la classe Rushhour pour qu'elle tienne compte de l'emplacement des roches.

3. Implémenter la fonction *`possible_rock_moves()`* qui permet de trouver tous les coups possibles de l'adversaire (avec la roche) à partir de l'état courant dans la classe Rushhour. L'adversaire ne peut ni mettre de roche sur la ligne 2 ni mettre deux roches consécutives sur la même ligne ou colonne.

Utiliser la fonction *testRocks()* pour vérifier que vous avez bien pris en compte les roches. Vous devriez avoir ces résultats:

```
État initial
[[b'-' b'-' b'v' b'j' b'j' b'j']
 [b'o' b'-' b'v' b'v' b'b' b'b']
 [b'o' b'r' b'r' b'v' b'-' b'-']
 [b'-' b'r' b'-' b'v' b'v' b'v']
 [b'v' b'r' b'n' b'n' b'-' b'b']
 [b'v' b'b' b'b' b'b' b'-' b'b']]
[[ True  True False False False False]
 [False  True False False False False]
 [False False False False  True  True]
 [ True False  True False False False]
 [False False False False  True False]
 [False False False False  True False]]


Roche 4-4
[[b'-' b'-' b'v' b'j' b'j' b'j']
 [b'o' b'-' b'v' b'v' b'b' b'b']
 [b'o' b'r' b'r' b'v' b'-' b'-']
 [b'-' b'r' b'-' b'v' b'v' b'v']
 [b'v' b'r' b'n' b'n' b'x' b'b']
 [b'v' b'b' b'b' b'b' b'-' b'b']]
[[ True  True False False False False]
 [False  True False False False False]
 [False False False False  True  True]
 [ True False  True False False False]
 [False False False False False False]
 [False False False False  True False]]


Roche 3-2
[[b'-' b'-' b'v' b'j' b'j' b'j']
 [b'o' b'-' b'v' b'v' b'b' b'b']
 [b'o' b'r' b'r' b'v' b'-' b'-']
 [b'-' b'r' b'x' b'v' b'v' b'v']
 [b'v' b'r' b'n' b'n' b'-' b'b']
 [b'v' b'b' b'b' b'b' b'-' b'b']]
[[ True  True False False False False]
 [False  True False False False False]
 [False False False False  True  True]
 [ True False False False False False]
 [False False False False False False]
 [False False False False  True False]]

```

Utiliser la fonction *testPossibleRockMoves()* pour vérifier que vous trouvez bien tous les coups possibles pour l'adversaire. Vous devriez trouver 7 et 3 mouvements possibles.

In [5]:
def testRocks():
    rh = Rushhour([True, False, True, False, False, True, False, True, False, True, False, True],
                 [2, 2, 3, 2, 3, 2, 2, 2, 2, 2, 2, 3],
                 [2, 2, 0, 0, 3, 1, 1, 3, 0, 4, 5, 5],
                 ["rouge", "vert clair", "jaune", "orange", "violet clair", "bleu ciel", "rose", "violet", "vert", "noir", "beige", "bleu"])
    s0 = State([1, 0, 3, 1, 1, 4, 3, 4, 4, 2, 4, 1])
    
    s1= s0.put_rock((4,4))
    s2 = s1.put_rock((3,2)) 
    
    print("État initial")
    rh.print_pretty_grid(s0)
    print(rh.free_pos)
    print('\n')
    
    print("Roche 4-4")
    rh.print_pretty_grid(s1)
    print(rh.free_pos)
    print('\n')
    
    print("Roche 3-2")
    rh.print_pretty_grid(s2)
    print(rh.free_pos)
    print('\n')

testRocks()

État initial
[[b'-' b'-' b'v' b'j' b'j' b'j']
 [b'o' b'-' b'v' b'v' b'b' b'b']
 [b'o' b'r' b'r' b'v' b'-' b'-']
 [b'-' b'r' b'-' b'v' b'v' b'v']
 [b'v' b'r' b'n' b'n' b'-' b'b']
 [b'v' b'b' b'b' b'b' b'-' b'b']]
[[ True  True False False False False]
 [False  True False False False False]
 [False False False False  True  True]
 [ True False  True False False False]
 [False False False False  True False]
 [False False False False  True False]]


Roche 4-4
[[b'-' b'-' b'v' b'j' b'j' b'j']
 [b'o' b'-' b'v' b'v' b'b' b'b']
 [b'o' b'r' b'r' b'v' b'-' b'-']
 [b'-' b'r' b'-' b'v' b'v' b'v']
 [b'v' b'r' b'n' b'n' b'x' b'b']
 [b'v' b'b' b'b' b'b' b'-' b'b']]
[[ True  True False False False False]
 [False  True False False False False]
 [False False False False  True  True]
 [ True False  True False False False]
 [False False False False False False]
 [False False False False  True False]]


Roche 3-2
[[b'-' b'-' b'v' b'j' b'j' b'j']
 [b'o' b'-' b'v' b'v' b'b' b'b']
 [b'o' b'r' b'r' b'v' b'-' b'

In [6]:
def testPossibleRockMoves():
    rh = Rushhour([True, False, True, False, False, True, False, True, False, True, False, True],
                 [2, 2, 3, 2, 3, 2, 2, 2, 2, 2, 2, 3],
                 [2, 2, 0, 0, 3, 1, 1, 3, 0, 4, 5, 5],
                 ["rouge", "vert clair", "jaune", "orange", "violet clair", "bleu ciel", "rose", "violet", "vert", "noir", "beige", "bleu"])
    s = State([1, 0, 3, 1, 1, 4, 3, 4, 4, 2, 4, 1])
    sols = rh.possible_rock_moves(s)
    print(len(sols))
    s1 = s.put_rock((3,4))
    sols = rh.possible_rock_moves(s1)
    print(len(sols))

testPossibleRockMoves()
  

7
3


## 2. Implémentation d'une recherche minimax (35pts)
L'algorithme minimax est l'implémentation la plus commune pour résoudre un jeu adversarielle avec des mouvements limités définis. Cet algorithme est basé sur le principe de construire un arbre avec les différents états possibles selon les actions de chaque joueur (AI et adversaire). L'algorithme veut maximiser les étages où le AI joue et minimiser les étages où l'adversaire joue. Dans ce cas-ci, l'AI déplace les voitures et l'adversaire place les roches. Dans le cadre du TP, l'algorithme n'explore pas jusqu'aux feuilles de l'arbre. Il s'arrête à une profondeur donnée.

La résolution du problème est faite par la classe **MiniMaxSearch** qui contient les champs suivants:

- `rushhour`: la classe Rushhour qui contient les informations sur la grille;
- `state`: la classe State avec l'état actuel de la grille;
- `search_depth`: la profondeur maximale d'exploration de l'algorithme;


In [7]:
class MiniMaxSearch:
    def __init__(self, rushHour, initial_state, search_depth):
        self.rushhour = rushHour
        self.state = initial_state
        self.search_depth = search_depth

    def remove_reverse_move(self, state, visited):
        children = self.rushhour.possible_moves(state)
        new_moves = []
        for child in children:
            if child.d != visited[child.c]:
                new_moves.append(child)

        return new_moves

    def minimax_1(self, current_depth, current_state, visited):
        # TODO
        children = self.remove_reverse_move(current_state, visited)

        if current_depth == self.search_depth or current_state.success():
            current_state.score_state(self.rushhour)
            return current_state

        values = []
        for child_state in children:
            values.append(self.minimax_1(current_depth + 1, child_state, visited))

        best_max = max(values)

        return best_max

    # Retourner le score au lieu d'un state, quand t au depth de niveau 1, tu choisis le meilleur score
    # AI, Roche, AI, Roche, ...
    def minimax_2(self, current_depth, current_state, is_max, visited, nodes):
        if current_depth == self.search_depth or current_state.success():
            nodes += 1
            current_state.score_state(self.rushhour)
            return current_state, nodes

        if is_max:
            max_values = []
            children = self.remove_reverse_move(current_state, visited)
            for child_state in children:
                child_value, nodes = self.minimax_2(current_depth + 1, child_state, False, visited, nodes)
                nodes += 1
                max_values.append(child_value)
            return max(max_values), nodes
        else:
            min_values = []
            children = self.rushhour.possible_rock_moves(current_state)
            for child_state in children:
                child_value, nodes = self.minimax_2(current_depth + 1, child_state, True, visited, nodes)
                nodes += 1
                min_values.append(child_value)
            return min(min_values), nodes

    def minimax_pruning(self, current_depth, current_state, is_max, alpha, beta, visited, nodes):
        # TODO
        if current_depth == self.search_depth or current_state.success():
            nodes += 1
            current_state.score_state(self.rushhour)
            return current_state, nodes

        if is_max:
            max_values = []
            children = self.remove_reverse_move(current_state, visited)
            for child_state in children:
                child_value, nodes = self.minimax_pruning(current_depth + 1, child_state, False, alpha, beta, visited, nodes)
                nodes += 1
                max_values.append(child_value)

                if alpha >= child_value.score:
                    break
            return max(max_values), nodes
        else:
            min_values = []
            children = self.rushhour.possible_rock_moves(current_state)
            for child_state in children:
                nodes += 1
                child_value = self.minimax_pruning(current_depth + 1, child_state, True, alpha, beta, visited, nodes)[0]
                min_values.append(child_value)

                if child_value.score >= beta:
                    break
            return min(min_values), nodes

    def expectimax(self, current_depth, current_state, is_max, visited, ai_vision):
        # TODO
        if current_depth == self.search_depth or current_state.success():
            current_state.score_state(self.rushhour)
            return current_state

        if is_max:
            children = self.remove_reverse_move(current_state, visited)
            if len(children) == 0:
                children = self.rushhour.possible_moves(current_state)

            max_values = []
            for child_state in children:
                max_values.append(self.expectimax(current_depth + 1, child_state, False, visited, ai_vision))

            return max(max_values)
        else:
            min_values = []
            children = self.rushhour.possible_rock_moves(current_state)
            for child_state in children:
                min_values.append(self.expectimax(current_depth + 1, child_state, True, visited, ai_vision))
            sum = 0
            for value in min_values:
                sum += abs(value.score)

            if ai_vision == 'None':
                return random.choice(min_values)
            elif ai_vision == 'aleatoire':
                return random.choice(min_values)
            elif ai_vision == 'pessimiste':
                min_values.sort()
                proba = []
                for value in min_values:
                    proba.append(abs(value.score / sum))
                return np.random.choice(min_values, p=proba)
            elif ai_vision == 'optimiste':
                min_values.sort(reverse=True)
                proba = []
                for value in min_values:
                    proba.append(abs(value.score / sum))
                return np.random.choice(min_values, p=proba)

    def decide_best_move_1(self, state, visited):
        # TODO
        return self.minimax_1(0, state, visited)
        # return state.move(best_child.c, best_child.d)

    def decide_best_move_2(self, state, is_max, visited, nodes):
        # TODO
        return self.minimax_2(0, state, is_max, visited, nodes)

    def decide_best_move_pruning(self, state, is_max, visited, nodes):
        # TODO
        return self.minimax_pruning(0, state, is_max, -math.inf, math.inf, visited, nodes)

    def decide_best_move_expectimax(self, state, is_max, visited, ai_vision):
        # TODO
        move = self.expectimax(0, state, is_max, visited, ai_vision)
        return move

    def solve(self, state, is_singleplayer, algo_type=None, ai_vision="aleatoire"):
        # TODO
        visited = [0] * self.rushhour.nbcars
        compteur = 0
        nodes = 0
        if algo_type is None:
            if is_singleplayer:
                while not state.success():
                    compteur += 1
                    state = self.decide_best_move_1(state, visited)
                    visited[state.c] = state.d * -1
                    print("Mouvement: " + str(compteur))
                    self.print_move(True, state)
            else:
                isMax = True
                i = 1
                while not state.success():
                    compteur += 1
                    state, nodes = self.decide_best_move_2(state, isMax, visited, nodes)
                    i, visited = self.print_previous_depth(state, compteur, i, visited)
                    isMax = not isMax
                print("Visited " + str(nodes))
        elif algo_type == "pruning":
            isMax = True
            i = 1
            while not state.success():
                compteur += 1
                state, nodes = self.decide_best_move_pruning(state, isMax, visited, nodes)
                i, visited = self.print_previous_depth(state, compteur, i, visited)
                isMax = not isMax
            print("Visited " + str(nodes) + " nodes")
        elif algo_type == "expectimax":
            isMax = True
            i = 1
            while not state.success():
                compteur += 1
                state = self.decide_best_move_expectimax(state, isMax, visited, ai_vision)
                i, visited = self.print_previous_depth(state, compteur, i, visited)
                isMax = not isMax

        return state

    def print_previous_depth(self, state, compteur, i, visited):
        if compteur % 2 == 1:
            # Joueur
            first_state = state.prev
            visited[first_state.c] = first_state.d * -1
            print("Mouvement: " + str(i))
            self.print_move(True, first_state)
            if first_state.success():
                return None, None

            # Roche
            print("Mouvement: " + str(i + 1))
            self.print_move(False, state)

            # Joueur
            visited[state.c] = state.d * -1
            print("Mouvement: " + str(i + 2))
            self.print_move(True, state)
        else:
            # Roche
            first_state = state.prev
            print("Mouvement: " + str(i))
            self.print_move(False, first_state)

            # Joueur
            visited[state.c] = state.d * -1
            print("Mouvement: " + str(i + 1))
            self.print_move(True, state)
            if state.success():
                return None, None

            # Roche
            print("Mouvement: " + str(i + 2))
            self.print_move(False, state)

        return i + 3, visited

    def print_move(self, is_max, state):
        # TODO
        if is_max:
            index_car = state.c
            if self.rushhour.horiz[index_car]:
                if state.d == +1:
                    print("Voiture " + self.rushhour.color[index_car] + " vers la droite")
                else:
                    print("Voiture " + self.rushhour.color[index_car] + " vers la gauche")
            else:
                if state.d == +1:
                    print("Voiture " + self.rushhour.color[index_car] + " vers le bas")
                else:
                    print("Voiture " + self.rushhour.color[index_car] + " vers le haut")
        else:
            print("Roche dans la case " + str(state.rock[0]) + "-" + str(state.rock[1]))

        return None

### 2.1 Implémentation simple
1. Implémenter la fonction *`score_state()`* de la classe **State**. Elle affecte la valeur de l'état à son paramètre score. L'état n'est pas nécessairement final. Utiliser l'heuristique qui vous semble la plus pertinente.

2. Implémenter la fonction  *`minimax_1()`*. Cette fonction contient la logique de l'algorithme minimax pour un seul joueur et retourne le meilleur coup à prendre à partir de l'état courant.

3. Implémenter la fonction *`decide_best_move_1()`*. Cette fonction trouve et exécute le meilleur coup pour une partie à un joueur.

4. Implémenter la fonction *`solve()`*. Cette fonction résout un problème de Rush Hour avec le nombre minimal de coups.

4. Implémenter la fonction *`print_move()`* pour imprimer le coup fait. Ex. "Voiture orange vers le haut" ou "Roche dans la case 2-1".

**N.B**: Vous pouvez modifier les déclarations des fonctions ainsi qu'ajouter des fonctions tant que l'algorithme fonctionne.

Utiliser la fonction _test_print_move()_ pour vérifier votre implémentation de _print_move()_. 

In [8]:
def test_print_move():
    rh = Rushhour([True], [2], [2], ["rouge"])
    s = State([0])
    s = s.put_rock((3,1)) # Roche dans la case 3-1
    s = s.move(0, 1) # Voiture rouge vers la droite

    algo = MiniMaxSearch(rh, s, 1)
    algo.print_move(True, s)
    algo.print_move(False, s)

test_print_move()

Voiture rouge vers la droite
Roche dans la case 3-1


Pour vous aider dans l'implémentation de votre heuristique, 3 exemples de problèmes de Rush Hour sont fournis.

In [9]:
# Solution optimale: 9 moves
rh = Rushhour([True, False, False, False, True],
                 [2, 3, 2, 3, 3],
                 [2, 4, 5, 1, 5],
                 ["rouge", "vert", "bleu", "orange", "jaune"])
s = State([1, 0, 1, 3, 2])
algo = MiniMaxSearch(rh, s,1) 
algo.rushhour.init_positions(s)
print(algo.rushhour.free_pos)
algo.solve(s, True)
%time

[[ True  True  True  True False  True]
 [ True  True  True  True False False]
 [ True False False  True False False]
 [ True False  True  True  True  True]
 [ True False  True  True  True  True]
 [ True False False False False  True]]
Mouvement: 1
Voiture rouge vers la droite
Mouvement: 2
Voiture bleu vers le haut
Mouvement: 3
Voiture vert vers le bas
Mouvement: 4
Voiture vert vers le bas
Mouvement: 5
Voiture orange vers le haut
Mouvement: 6
Voiture jaune vers la gauche
Mouvement: 7
Voiture vert vers le bas
Mouvement: 8
Voiture rouge vers la droite
Mouvement: 9
Voiture rouge vers la droite
Wall time: 0 ns


In [10]:
# solution optimale: 16 moves
rh = Rushhour([True, True, False, False, True, True, False, False],
                 [2, 2, 3, 2, 3, 2, 3, 3],
                 [2, 0, 0, 0, 5, 4, 5, 3],
                 ["rouge", "vert", "mauve", "orange", "emeraude", "lime", "jaune", "bleu"])
s = State([1, 0, 1, 4, 2, 4, 0, 1])
algo = MiniMaxSearch(rh, s, 1) 
algo.rushhour.init_positions(s)
print(algo.rushhour.free_pos)
algo.solve(s, True) 
%time

[[False False  True  True  True False]
 [False  True  True False  True False]
 [False False False False  True False]
 [False  True  True False  True  True]
 [False  True  True  True False False]
 [False  True False False False  True]]
Mouvement: 1
Voiture jaune vers le bas
Mouvement: 2
Voiture emeraude vers la gauche
Mouvement: 3
Voiture lime vers la gauche
Mouvement: 4
Voiture jaune vers le bas
Mouvement: 5
Voiture jaune vers le bas
Mouvement: 6
Voiture vert vers la droite
Mouvement: 7
Voiture mauve vers le haut
Mouvement: 8
Voiture lime vers la gauche
Mouvement: 9
Voiture lime vers la gauche
Mouvement: 10
Voiture bleu vers le bas
Mouvement: 11
Voiture orange vers le haut
Mouvement: 12
Voiture emeraude vers la gauche
Mouvement: 13
Voiture bleu vers le bas
Mouvement: 14
Voiture rouge vers la droite
Mouvement: 15
Voiture rouge vers la droite
Mouvement: 16
Voiture rouge vers la droite
Wall time: 0 ns


In [11]:
# solution optimale: 14 moves
rh = Rushhour([True, False, True, False, False, False, True, True, False, True, True],
                 [2, 2, 3, 2, 2, 3, 3, 2, 2, 2, 2],
                 [2, 0, 0, 3, 4, 5, 3, 5, 2, 5, 4],
                 ["rouge", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10"])
s = State([0, 0, 3, 1, 2, 1, 0, 0, 4, 3, 4])
algo = MiniMaxSearch(rh, s,1)
algo.rushhour.init_positions(s)
print(algo.rushhour.free_pos)
algo.solve(s, True)
%time

[[False  True  True False False False]
 [False  True  True False  True False]
 [False False  True False False False]
 [False False False  True False False]
 [ True  True False  True False False]
 [False False False False False  True]]
Mouvement: 1
Voiture rouge vers la droite
Mouvement: 2
Voiture 10 vers la gauche
Mouvement: 3
Voiture 5 vers le bas
Mouvement: 4
Voiture 5 vers le bas
Mouvement: 5
Voiture 2 vers la gauche
Mouvement: 6
Voiture 2 vers la gauche
Mouvement: 7
Voiture 4 vers le haut
Mouvement: 8
Voiture 4 vers le haut
Mouvement: 9
Voiture 1 vers le bas
Mouvement: 10
Voiture 2 vers la gauche
Mouvement: 11
Voiture 3 vers le haut
Mouvement: 12
Voiture rouge vers la droite
Mouvement: 13
Voiture rouge vers la droite
Mouvement: 14
Voiture rouge vers la droite
Wall time: 0 ns


### 2.2 Implémentation adversarielle

1. Implémenter la fonction  *`minimax_2()`*. Cette fonction contient la logique de l'algorithme minimax pour deux joueurs et retourne le meilleur coup à prendre à partir de l'état courant.

2. Implémenter la fonction *`decide_best_move_2()`*. Cette fonction trouve et exécute le meilleur coup pour une partie à deux joueurs.

3. Modifier la fonction *`solve()`* pour qu'elle puisse résoudre des problèmes à deux joueurs.

4. Modifier la fonction *`score_state()`* de la classe **State** si elle ne tient pas déjà compte des roches.

**N.B**: Vous pouvez modifier les déclarations des fonctions ainsi qu'ajouter des fonctions tant que l'algorithme fonctionne.

 Le nombre donné de coups pour les 3 exemples suivants correspond à une partie sans adversaire. Le nombre de coups de plus dépend de l'implémentation de votre heuristique.

In [ ]:
NOTE: Un mouvement correspond à un tour. Exemple: 17 mouvements correspond à 9 mouvements de l'AI

In [12]:
# Solution optimale: 9 moves
rh = Rushhour([True, False, False, False, True],
                 [2, 3, 2, 3, 3],
                 [2, 4, 5, 1, 5],
                 ["rouge", "vert", "bleu", "orange", "jaune"])
s = State([1, 0, 1, 3, 2])
algo = MiniMaxSearch(rh, s,3)
algo.rushhour.init_positions(s)
print(algo.rushhour.free_pos)
algo.solve(s, False)
%time

[[ True  True  True  True False  True]
 [ True  True  True  True False False]
 [ True False False  True False False]
 [ True False  True  True  True  True]
 [ True False  True  True  True  True]
 [ True False False False False  True]]
Mouvement: 1
Voiture vert vers le bas
Mouvement: 2
Roche dans la case 5-5
Mouvement: 3
Voiture rouge vers la droite
Mouvement: 4
Roche dans la case 0-0
Mouvement: 5
Voiture bleu vers le haut
Mouvement: 6
Roche dans la case 5-5
Mouvement: 7
Voiture orange vers le haut
Mouvement: 8
Roche dans la case 4-4
Mouvement: 9
Voiture jaune vers la gauche
Mouvement: 10
Roche dans la case 0-0
Mouvement: 11
Voiture vert vers le bas
Mouvement: 12
Roche dans la case 5-5
Mouvement: 13
Voiture vert vers le bas
Mouvement: 14
Roche dans la case 1-1
Mouvement: 15
Voiture rouge vers la droite
Mouvement: 16
Roche dans la case 5-0
Mouvement: 17
Voiture rouge vers la droite
Visited 6232
Wall time: 0 ns


In [13]:
# solution optimale: 16 moves
rh = Rushhour([True, True, False, False, True, True, False, False],
                 [2, 2, 3, 2, 3, 2, 3, 3],
                 [2, 0, 0, 0, 5, 4, 5, 3],
                 ["rouge", "vert", "mauve", "orange", "emeraude", "lime", "jaune", "bleu"])
s = State([1, 0, 1, 4, 2, 4, 0, 1])
algo = MiniMaxSearch(rh, s, 3) 
algo.rushhour.init_positions(s)
print(algo.rushhour.free_pos)
algo.solve(s, False) 
%time

[[False False  True  True  True False]
 [False  True  True False  True False]
 [False False False False  True False]
 [False  True  True False  True  True]
 [False  True  True  True False False]
 [False  True False False False  True]]
Mouvement: 1
Voiture jaune vers le bas
Mouvement: 2
Roche dans la case 4-3
Mouvement: 3
Voiture vert vers la droite
Mouvement: 4
Roche dans la case 0-0
Mouvement: 5
Voiture lime vers la gauche
Mouvement: 6
Roche dans la case 5-1
Mouvement: 7
Voiture jaune vers le bas
Mouvement: 8
Roche dans la case 4-2
Mouvement: 9
Voiture jaune vers le bas
Mouvement: 10
Roche dans la case 5-1
Mouvement: 11
Voiture mauve vers le haut
Mouvement: 12
Roche dans la case 0-3
Mouvement: 13
Voiture lime vers la gauche
Mouvement: 14
Roche dans la case 5-1
Mouvement: 15
Voiture lime vers la gauche
Mouvement: 16
Roche dans la case 0-3
Mouvement: 17
Voiture emeraude vers la gauche
Mouvement: 18
Roche dans la case 5-4
Mouvement: 19
Voiture orange vers le haut
Mouvement: 20
Roche dans

In [14]:
# solution optimale: 14 moves
rh = Rushhour([True, False, True, False, False, False, True, True, False, True, True],
                 [2, 2, 3, 2, 2, 3, 3, 2, 2, 2, 2],
                 [2, 0, 0, 3, 4, 5, 3, 5, 2, 5, 4],
                 ["rouge", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10"])
s = State([0, 0, 3, 1, 2, 1, 0, 0, 4, 3, 4])
algo = MiniMaxSearch(rh, s,3)
algo.rushhour.init_positions(s)
print(algo.rushhour.free_pos)
algo.solve(s, False)
%time

[[False  True  True False False False]
 [False  True  True False  True False]
 [False False  True False False False]
 [False False False  True False False]
 [ True  True False  True False False]
 [False False False False False  True]]
Mouvement: 1
Voiture rouge vers la droite
Mouvement: 2
Roche dans la case 5-5
Mouvement: 3
Voiture 2 vers la gauche
Mouvement: 4
Roche dans la case 0-1
Mouvement: 5
Voiture 10 vers la gauche
Mouvement: 6
Roche dans la case 5-5
Mouvement: 7
Voiture 5 vers le bas
Mouvement: 8
Roche dans la case 3-3
Mouvement: 9
Voiture 4 vers le haut
Mouvement: 10
Roche dans la case 0-1
Mouvement: 11
Voiture 5 vers le bas
Mouvement: 12
Roche dans la case 3-3
Mouvement: 13
Voiture 2 vers la gauche
Mouvement: 14
Roche dans la case 0-4
Mouvement: 15
Voiture 1 vers le bas
Mouvement: 16
Roche dans la case 1-5
Mouvement: 17
Voiture 4 vers le haut
Mouvement: 18
Roche dans la case 3-3
Mouvement: 19
Voiture 2 vers la gauche
Mouvement: 20
Roche dans la case 0-5
Mouvement: 21
Voiture 

## 3. Élagage Alpha-Beta (15pts)
Pour accélérer la prise de décision, un élagage est effectué sur l'arbre. Cela permet de ne pas parcourir tout l'arbre. L'élagage alpha-beta est l'implémentation la plus fréquente pour résoudre ce genre de problème. En passant des valeurs d’alpha et de bêta récursivement, l'algorithme connaît la meilleure valeur trouvée à date. Les branches jugées non intéressantes, c'est-à-dire qu'elles ne peuvent pas produire un meilleur résultat que celui déjà trouvé, sont coupées dans le processus.

 
### 3.1 Implémentation

1. Implémenter la fonction *`minimax_pruning()`* pour qu'elle n'explore pas des branches inutiles lors d'une partie à deux joueurs.
2. Implémenter la fonction *`decide_best_move_pruning()`*. Cette fonction trouve et exécute le meilleur coup pour une partie à deux joueurs.
3. Modifier la fonction *`solve()`* pour qu'elle utilise cette nouvelle méthode.
4. Vérifier que cette nouvelle méthode visite bel et bien moins de noeuds. **Montrer cette information dans un tableau**.

In [15]:
# solution optimale: 9 moves
rh = Rushhour([True, False, False, False, True],
                 [2, 3, 2, 3, 3],
                 [2, 4, 5, 1, 5],
                 ["rouge", "vert", "bleu", "orange", "jaune"])
s = State([1, 0, 1, 3, 2])
algo = MiniMaxSearch(rh, s,3)
algo.rushhour.init_positions(s)
print(algo.rushhour.free_pos)
algo.solve(s, False, "pruning")
%time

[[ True  True  True  True False  True]
 [ True  True  True  True False False]
 [ True False False  True False False]
 [ True False  True  True  True  True]
 [ True False  True  True  True  True]
 [ True False False False False  True]]
Mouvement: 1
Voiture vert vers le bas
Mouvement: 2
Roche dans la case 5-5
Mouvement: 3
Voiture rouge vers la droite
Mouvement: 4
Roche dans la case 0-0
Mouvement: 5
Voiture bleu vers le haut
Mouvement: 6
Roche dans la case 5-5
Mouvement: 7
Voiture orange vers le haut
Mouvement: 8
Roche dans la case 4-4
Mouvement: 9
Voiture jaune vers la gauche
Mouvement: 10
Roche dans la case 0-0
Mouvement: 11
Voiture vert vers le bas
Mouvement: 12
Roche dans la case 5-5
Mouvement: 13
Voiture vert vers le bas
Mouvement: 14
Roche dans la case 1-1
Mouvement: 15
Voiture rouge vers la droite
Mouvement: 16
Roche dans la case 5-0
Mouvement: 17
Voiture rouge vers la droite
Visited 260 nodes
Wall time: 0 ns


In [16]:
# solution optimale: 16 moves
rh = Rushhour([True, True, False, False, True, True, False, False],
                 [2, 2, 3, 2, 3, 2, 3, 3],
                 [2, 0, 0, 0, 5, 4, 5, 3],
                 ["rouge", "vert", "mauve", "orange", "emeraude", "lime", "jaune", "bleu"])
s = State([1, 0, 1, 4, 2, 4, 0, 1])
algo = MiniMaxSearch(rh, s, 3) 
algo.rushhour.init_positions(s)
print(algo.rushhour.free_pos)
algo.solve(s, False, "pruning") 
%time

[[False False  True  True  True False]
 [False  True  True False  True False]
 [False False False False  True False]
 [False  True  True False  True  True]
 [False  True  True  True False False]
 [False  True False False False  True]]
Mouvement: 1
Voiture jaune vers le bas
Mouvement: 2
Roche dans la case 4-3
Mouvement: 3
Voiture vert vers la droite
Mouvement: 4
Roche dans la case 0-0
Mouvement: 5
Voiture lime vers la gauche
Mouvement: 6
Roche dans la case 5-1
Mouvement: 7
Voiture jaune vers le bas
Mouvement: 8
Roche dans la case 4-2
Mouvement: 9
Voiture jaune vers le bas
Mouvement: 10
Roche dans la case 5-1
Mouvement: 11
Voiture mauve vers le haut
Mouvement: 12
Roche dans la case 0-3
Mouvement: 13
Voiture lime vers la gauche
Mouvement: 14
Roche dans la case 5-1
Mouvement: 15
Voiture lime vers la gauche
Mouvement: 16
Roche dans la case 0-3
Mouvement: 17
Voiture emeraude vers la gauche
Mouvement: 18
Roche dans la case 5-4
Mouvement: 19
Voiture orange vers le haut
Mouvement: 20
Roche dans

In [17]:
# solution optimale: 14 moves
rh = Rushhour([True, False, True, False, False, False, True, True, False, True, True],
                 [2, 2, 3, 2, 2, 3, 3, 2, 2, 2, 2],
                 [2, 0, 0, 3, 4, 5, 3, 5, 2, 5, 4],
                 ["rouge", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10"])
s = State([0, 0, 3, 1, 2, 1, 0, 0, 4, 3, 4])
algo = MiniMaxSearch(rh, s,3)
algo.rushhour.init_positions(s)
print(algo.rushhour.free_pos)
algo.solve(s, False, "pruning")
%time

[[False  True  True False False False]
 [False  True  True False  True False]
 [False False  True False False False]
 [False False False  True False False]
 [ True  True False  True False False]
 [False False False False False  True]]
Mouvement: 1
Voiture rouge vers la droite
Mouvement: 2
Roche dans la case 5-5
Mouvement: 3
Voiture 2 vers la gauche
Mouvement: 4
Roche dans la case 0-1
Mouvement: 5
Voiture 10 vers la gauche
Mouvement: 6
Roche dans la case 5-5
Mouvement: 7
Voiture 5 vers le bas
Mouvement: 8
Roche dans la case 3-3
Mouvement: 9
Voiture 4 vers le haut
Mouvement: 10
Roche dans la case 0-1
Mouvement: 11
Voiture 5 vers le bas
Mouvement: 12
Roche dans la case 3-3
Mouvement: 13
Voiture 2 vers la gauche
Mouvement: 14
Roche dans la case 0-4
Mouvement: 15
Voiture 1 vers le bas
Mouvement: 16
Roche dans la case 1-5
Mouvement: 17
Voiture 4 vers le haut
Mouvement: 18
Roche dans la case 3-3
Mouvement: 19
Voiture 2 vers la gauche
Mouvement: 20
Roche dans la case 0-5
Mouvement: 21
Voiture 

In [6]:
from IPython.display import HTML, display
import tabulate

table = [["", "Without pruning", "With pruning"],
         ["9 moves", 6232, 260],
         ["16 moves", 7158, 332],
         ["14 moves", 3852, 193]]

display(HTML(tabulate.tabulate(table, tablefmt='html')))

,Without pruning,With pruning
9 moves,6232,260
16 moves,7158,332
14 moves,3852,193


## 4. Expectimax (15 pts)
L'expectimax est une variante de la recherche minimax. Elle est utilisée lorsqu'il y a une composante de hasard lors de la prise de décision. Depuis le début du TP, il a été supposé que l'adversaire prenait toujours le coup optimal. Maintenant, l'adversaire va faire un coup aléatoirement. Pour résoudre ce cas-là, la recherche expectimax sera utilisée.

### 4.1 Implémentation 

1. Implémenter la fonction *`minimax_expectimax()`* pour qu'elle implémente l'algorithme de recherche. Considérer une probabilité égale pour tous les coups.
2. Implémenter la fonction *`decide_best_move_expectimax()`*. Cette fonction trouve et exécute le meilleur coup pour une partie avec un adversaire imprévisible.
3. Modifier la fonction *`solve()`* pour qu'elle utilise cette nouvelle méthode.
4. Modifier les valeurs de probabilités associées aux coups pour donner une vision optimiste et une vision pessimiste à l'AI. Comparer la performance du AI entre les 3 visions (aléatoire, optimiste, pessimiste). Attention l'adversaire continue à prendre ses décisions au hasard. **Montrer ces informations dans un tableau et les analyser**. 

In [26]:
# solution optimale: 9 moves
rh = Rushhour([True, False, False, False, True],
                 [2, 3, 2, 3, 3],
                 [2, 4, 5, 1, 5],
                 ["rouge", "vert", "bleu", "orange", "jaune"])
s = State([1, 0, 1, 3, 2])
algo = MiniMaxSearch(rh, s,3)
algo.rushhour.init_positions(s)
print(algo.rushhour.free_pos)
algo.solve(s, False, "expectimax", "optimiste")
%time

[[ True  True  True  True False  True]
 [ True  True  True  True False False]
 [ True False False  True False False]
 [ True False  True  True  True  True]
 [ True False  True  True  True  True]
 [ True False False False False  True]]
Mouvement: 1
Voiture bleu vers le haut
Mouvement: 2
Roche dans la case 1-3
Mouvement: 3
Voiture rouge vers la droite
Mouvement: 4
Roche dans la case 4-2
Mouvement: 5
Voiture vert vers le bas
Mouvement: 6
Roche dans la case 0-1
Mouvement: 7
Voiture orange vers le haut
Mouvement: 8
Roche dans la case 1-0
Mouvement: 9
Voiture jaune vers la gauche
Mouvement: 10
Roche dans la case 3-3
Mouvement: 11
Voiture vert vers le bas
Mouvement: 12
Roche dans la case 0-1
Mouvement: 13
Voiture vert vers le bas
Mouvement: 14
Roche dans la case 4-3
Mouvement: 15
Voiture rouge vers la droite
Mouvement: 16
Roche dans la case 1-2
Mouvement: 17
Voiture rouge vers la droite
Wall time: 0 ns


In [27]:
# solution optimale: 16 moves
rh = Rushhour([True, True, False, False, True, True, False, False],
                 [2, 2, 3, 2, 3, 2, 3, 3],
                 [2, 0, 0, 0, 5, 4, 5, 3],
                 ["rouge", "vert", "mauve", "orange", "emeraude", "lime", "jaune", "bleu"])
s = State([1, 0, 1, 4, 2, 4, 0, 1])
algo = MiniMaxSearch(rh, s, 3) 
algo.rushhour.init_positions(s)
print(algo.rushhour.free_pos)
algo.solve(s, False, "expectimax", "optimiste") 
%time

[[False False  True  True  True False]
 [False  True  True False  True False]
 [False False False False  True False]
 [False  True  True False  True  True]
 [False  True  True  True False False]
 [False  True False False False  True]]
Mouvement: 1
Voiture lime vers la gauche
Mouvement: 2
Roche dans la case 0-2
Mouvement: 3
Voiture jaune vers le bas
Mouvement: 4
Roche dans la case 1-4
Mouvement: 5
Voiture jaune vers le bas
Mouvement: 6
Roche dans la case 4-2
Mouvement: 7
Voiture jaune vers le bas
Mouvement: 8
Roche dans la case 0-5
Mouvement: 9
Voiture emeraude vers la gauche
Mouvement: 10
Roche dans la case 5-4
Mouvement: 11
Voiture vert vers la droite
Mouvement: 12
Roche dans la case 1-1
Mouvement: 13
Voiture lime vers la gauche
Mouvement: 14
Roche dans la case 3-2
Mouvement: 15
Voiture lime vers la gauche
Mouvement: 16
Roche dans la case 0-5
Mouvement: 17
Voiture mauve vers le haut
Mouvement: 18
Roche dans la case 4-4
Mouvement: 19
Voiture orange vers le haut
Mouvement: 20
Roche dans

In [28]:
# solution optimale: 14 moves
rh = Rushhour([True, False, True, False, False, False, True, True, False, True, True],
                 [2, 2, 3, 2, 2, 3, 3, 2, 2, 2, 2],
                 [2, 0, 0, 3, 4, 5, 3, 5, 2, 5, 4],
                 ["rouge", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10"])
s = State([0, 0, 3, 1, 2, 1, 0, 0, 4, 3, 4])
algo = MiniMaxSearch(rh, s,3)
algo.rushhour.init_positions(s)
print(algo.rushhour.free_pos)
algo.solve(s, False, "expectimax", "optimiste")
%time

[[False  True  True False False False]
 [False  True  True False  True False]
 [False False  True False False False]
 [False False False  True False False]
 [ True  True False  True False False]
 [False False False False False  True]]
Mouvement: 1
Voiture 2 vers la gauche
Mouvement: 2
Roche dans la case 1-2
Mouvement: 3
Voiture rouge vers la droite
Mouvement: 4
Roche dans la case 4-0
Mouvement: 5
Voiture 10 vers la gauche
Mouvement: 6
Roche dans la case 0-5
Mouvement: 7
Voiture 1 vers le bas
Mouvement: 8
Roche dans la case 1-1
Mouvement: 9
Voiture 5 vers le bas
Mouvement: 10
Roche dans la case 5-5
Mouvement: 11
Voiture 2 vers la gauche
Mouvement: 12
Roche dans la case 3-3
Mouvement: 13
Voiture 4 vers le haut
Mouvement: 14
Roche dans la case 0-5
Mouvement: 15
Voiture 4 vers le haut
Mouvement: 16
Roche dans la case 1-2
Mouvement: 17
Voiture 2 vers la gauche
Mouvement: 18
Roche dans la case 0-3
Mouvement: 19
Voiture 5 vers le bas
Mouvement: 20
Roche dans la case 1-5
Mouvement: 21
Voiture 

Pour bien comparer les 3 cas (aléatoire, optimiste et pessimiste), nous avons exécuté le code 3 fois pour chaque cas.
Une mouvement compte comme étant soit un deplacement d'une auto ou placer une roche

In [29]:
print("Cas aléatoire")

table = [["", "Execution 1", "Execution 2", "Execution 3"],
         ["9 moves", 17, 17, 17],
         ["16 moves", 41, 65, 33],
         ["14 moves", 33, 51, 33]]

display(HTML(tabulate.tabulate(table, tablefmt='html')))

Cas aléatoire


,Execution 1,Execution 2,Execution 3
9 moves,17,17,17
16 moves,41,65,33
14 moves,33,51,33


In [30]:
print("Cas optimiste")

table = [["", "Execution 1", "Execution 2", "Execution 3"],
         ["9 moves", 17, 17, 17],
         ["16 moves", 33, 33, 39],
         ["14 moves", 27, 27, 27]]

display(HTML(tabulate.tabulate(table, tablefmt='html')))

Cas optimiste


,Execution 1,Execution 2,Execution 3
9 moves,17,17,17
16 moves,33,33,39
14 moves,27,27,27


In [31]:
print("Cas pessimiste")

table = [["", "Execution 1", "Execution 2", "Execution 3"],
         ["9 moves", 17, 17, 17],
         ["16 moves", 81, 65, 41],
         ["14 moves", 33, 53, 41]]

display(HTML(tabulate.tabulate(table, tablefmt='html')))

Cas pessimiste


,Execution 1,Execution 2,Execution 3
9 moves,17,17,17
16 moves,81,65,41
14 moves,33,53,41


## 5. Comparaison (10pts)
Comparez la performance la recherche minimax avec la recherche expectimax lorsque l'adversaire est optimale et lorsque l'adversaire est aléatoire. Pour un adversaire aléatoire, prenez une vision neutre. Roulez plusieurs fois les algorithmes (vous pouvez aussi reprendre les tests du TP1) et notez si c'est une victoire ou une défaire pour l'AI ainsi que le nombre de coups moyen, minimal et maximal. Basez-vous sur le nombre de coups optimaux pour résoudre les problèmes pour déterminer un seuil de victoire et de défaite pour l'AI. Précisez votre seuil. **Mettez le tout dans un tableau et analysez vos résultats**. 

Pour le premier cas de test, qui est celui de 9 mouvements, le résultat est le même pour lorsque l'adversaire est aléatoire et lorsque l'adversaire est optimale. En se basant sur ces résultats, on peut conclure qu'une victoire correspond à une solution trouvée en 17 tours.  

In [10]:
print("Cas aléatoire - 9 moves")

table = [["", "Expectimax", "Minimax"],
         ["Execution 1", 17, 17],
         ["Execution 2", 17, 17],
         ["Execution 3", 17, 17],
         ["Execution 4", 17, 17],
         ["Execution 5", 17, 17]]

display(HTML(tabulate.tabulate(table, tablefmt='html')))

print("Nombre de coups moyen: 17")
print("Coup minimum: 17")
print("Coup maximum: 17")

print("\n\nCas optimiste - 9 moves")

table2 = [["", "Expectimax", "Minimax"],
         ["Execution 1", 17, 17],
         ["Execution 2", 17, 17],
         ["Execution 3", 17, 17],
         ["Execution 4", 17, 17],
         ["Execution 5", 17, 17]]

display(HTML(tabulate.tabulate(table2, tablefmt='html')))

print("Nombre de coups moyen: 17")
print("Coup minimum: 17")
print("Coup maximum: 17")

Cas aléatoire - 9 moves


,Expectimax,Minimax
Execution 1,17,17
Execution 2,17,17
Execution 3,17,17
Execution 4,17,17
Execution 5,17,17


Nombre de coups moyen: 17
Coup minimum: 17
Coup maximum: 17


Cas optimiste - 9 moves


,Expectimax,Minimax
Execution 1,17,17
Execution 2,17,17
Execution 3,17,17
Execution 4,17,17
Execution 5,17,17


Nombre de coups moyen: 17
Coup minimum: 17
Coup maximum: 17


Pour le deuxième cas de test, qui est celui de 16 mouvements, on peut voir une claire différence entre un adversaire aléatoire et un adversaire optimal. 
Pour l'algorithme minimax, on arrive à résoudre le problème en 39 tours. Lorsqu'on applique l'algorithme expectimax, on obtient un nombre de tour minimum de 33 et un nombre de tour maximum de 65. Dans la section précedente, nous avons vu que lorsque la vision du AI est pessimiste, on peut aller jusqu'à 85 tours!
En se basant sur ces résultats, on peut conclure qu'une victoire correspond à une solution trouvée en un maximum de 41 tours.

In [12]:
print("Cas aléatoire - 16 moves")

table = [["", "Expectimax", "Minimax"],
         ["Execution 1", 41, 39],
         ["Execution 2", 65, 39],
         ["Execution 3", 33, 39],
         ["Execution 4", 65, 39],
         ["Execution 5", 51, 39]]

display(HTML(tabulate.tabulate(table, tablefmt='html')))

print("Nombre de coups moyen: 51")
print("Coup minimum: 33")
print("Coup maximum: 65")

print("\n\nCas optimiste - 16 moves")

table2 = [["", "Expectimax", "Minimax"],
         ["Execution 1", 33, 39],
         ["Execution 2", 33, 39],
         ["Execution 3", 39, 39],
         ["Execution 4", 33, 39],
         ["Execution 5", 39, 39]]

display(HTML(tabulate.tabulate(table2, tablefmt='html')))

print("Nombre de coups moyen: 35")
print("Coup minimum: 33")
print("Coup maximum: 39")

Cas aléatoire - 16 moves


,Expectimax,Minimax
Execution 1,41,39
Execution 2,65,39
Execution 3,33,39
Execution 4,65,39
Execution 5,51,39


Nombre de coups moyen: 51
Coup minimum: 33
Coup maximum: 65


Cas optimiste - 16 moves


,Expectimax,Minimax
Execution 1,33,39
Execution 2,33,39
Execution 3,39,39
Execution 4,33,39
Execution 5,39,39


Nombre de coups moyen: 35
Coup minimum: 33
Coup maximum: 39


Pour le troisième cas de test, qui est celui de 14 mouvements, on peut voir une claire différence entre un adversaire aléatoire et un adversaire optimal. 

Pour l'algorithme minimax, on arrive à résoudre le problème en 27 tours. 

Lorsqu'on applique l'algorithme expectimax avec une vision du AI aléatoire, on obtient un nombre de tour minimum de 33 et un nombre de tour maximum de 51. Lorsque la vision du AI est optimale, on obtient toujours un résultat de 27 tours.

En se basant sur ces résultats, on peut conclure qu'une victoire correspond à une solution trouvée en un maximum de 33 tours.

In [13]:
print("Cas aléatoire - 14 moves")

table = [["", "Expectimax", "Minimax"],
         ["Execution 1", 33, 27],
         ["Execution 2", 51, 27],
         ["Execution 3", 33, 27],
         ["Execution 4", 33, 27],
         ["Execution 5", 41, 27]]

display(HTML(tabulate.tabulate(table, tablefmt='html')))

print("Nombre de coups moyen: 38")
print("Coup minimum: 33")
print("Coup maximum: 51")

print("\n\nCas optimiste - 14 moves")

table2 = [["", "Expectimax", "Minimax"],
         ["Execution 1", 27, 27],
         ["Execution 2", 27, 27],
         ["Execution 3", 27, 27],
         ["Execution 4", 27, 27],
         ["Execution 5", 27, 27]]

display(HTML(tabulate.tabulate(table2, tablefmt='html')))

print("Nombre de coups moyen: 27")
print("Coup minimum: 27")
print("Coup maximum: 27")

Cas aléatoire - 14 moves


,Expectimax,Minimax
Execution 1,33,27
Execution 2,51,27
Execution 3,33,27
Execution 4,33,27
Execution 5,41,27


Nombre de coups moyen: 38
Coup minimum: 33
Coup maximum: 51


Cas optimiste - 14 moves


,Expectimax,Minimax
Execution 1,27,27
Execution 2,27,27
Execution 3,27,27
Execution 4,27,27
Execution 5,27,27


Nombre de coups moyen: 27
Coup minimum: 27
Coup maximum: 27
